In [66]:
import numpy as np
import scipy.stats
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [13]:
data = pd.read_excel(r"/home/adam/Downloads/Stat online ANOVA 2019 data.xlsx", header=1, index_col=0)

In [14]:
data

,Aisle,End-Cap,Checkout Lane
Store,,,
1,738,712,833
2,726,685,795
3,722,725,782
4,715,699,754
5,767,632,772


In [22]:
# Too easy
scipy.stats.f_oneway(data['Aisle'], data['End-Cap'], data['Checkout Lane'])

F_onewayResult(statistic=13.55313813987579, pvalue=0.0008348504608853334)

In [24]:
# Much better
n_a = n_ec = n_cl = 5

xbar_a = np.mean(data['Aisle'])
xbar_ec = np.mean(data['End-Cap'])
xbar_cl = np.mean(data['Checkout Lane'])

grand_mean = (n_a*xbar_a + n_ec*xbar_ec + n_cl*xbar_cl) / 15

In [44]:
sstr = (n_a*(xbar_a - grand_mean)**2 + n_ec*(xbar_ec - grand_mean)**2 + n_cl*(xbar_cl - grand_mean)**2)
sse = np.sum((data['Aisle'] - xbar_a)**2 + (data['End-Cap'] - xbar_ec)**2 + (data['Checkout Lane'] - xbar_cl)**2)

n = 15
c = 3

In [54]:
mstr = sstr/(c-1)
mse = sse/(n-c)

In [55]:
# Critical F_(0.05, 2, 12) is 3.885
f = mstr/mse
f

13.553138139875792

# Tukey Test

In [65]:
df = data.unstack().reset_index(name='value').rename(columns = {'level_0': 'Treatment'})
df

,Treatment,Store,value
0,Aisle,1,738
1,Aisle,2,726
2,Aisle,3,722
3,Aisle,4,715
4,Aisle,5,767
5,End-Cap,1,712
6,End-Cap,2,685
7,End-Cap,3,725
8,End-Cap,4,699
9,End-Cap,5,632


In [67]:
# Easy Way
tukey = pairwise_tukeyhsd(endog=df['value'], groups=df['Treatment'], alpha=0.05)
print(tukey)

         Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1        group2    meandiff p-adj    lower    upper   reject
---------------------------------------------------------------------
        Aisle Checkout Lane     53.6  0.034    4.0251 103.1749   True
        Aisle       End-Cap    -43.0 0.0921  -92.5749   6.5749  False
Checkout Lane       End-Cap    -96.6  0.001 -146.1749 -47.0251   True
---------------------------------------------------------------------


In [68]:
# Hard Way
q = 3.77
tukey_val = q*np.sqrt(mse / 5)

In [69]:
tukey_val

49.56080485625713

In [73]:
xbar_a - xbar_ec # Smaller than tukey val - not significant

43.0

In [74]:
xbar_a - xbar_cl # Larger than tukey val magnitude- significant

-53.60000000000002

In [75]:
xbar_cl - xbar_ec # Larger than tukey val magnitude - significant

96.60000000000002

The checkout lane is significantly different than the aisle and the end-cap